In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import numpy as np

# Wire impedance values
impedance_data = {
    'Copper': {
        '500 kcmil': 0.022,
        '350 kcmil': 0.031,
        '4/0 AWG': 0.062,
        '2/0 AWG': 0.098,
        '1/0 AWG': 0.124,
        '1 AWG': 0.156,
        '2 AWG': 0.198
    },
    'Aluminum': {
        '500 kcmil': 0.036,
        '350 kcmil': 0.050,
        '4/0 AWG': 0.096,
        '2/0 AWG': 0.150,
        '1/0 AWG': 0.190,
        '1 AWG': 0.232,
        '2 AWG': 0.294
    }
}

# Define right-aligned layout for description placement
layout = widgets.Layout(width='250px', description_tooltip='')

# Widgets with right-side descriptions
config = widgets.Dropdown(options=[('Three-Phase', '3ph'), ('Single-Phase', '1ph')], value='3ph', description='Config:', layout=layout, style={'description_width': 'initial'})
kva = widgets.FloatText(value=75, description='kVA:', layout=layout, style={'description_width': 'initial'})
voltage = widgets.FloatText(value=480, description='Voltage (V):', layout=layout, style={'description_width': 'initial'})
impedance = widgets.FloatText(value=5.75, description='%Z Transformer:', layout=layout, style={'description_width': 'initial'})
material = widgets.Dropdown(options=['Copper', 'Aluminum'], value='Copper', description='Conductor:', layout=layout, style={'description_width': 'initial'})
conductor_size = widgets.Dropdown(options=list(impedance_data['Copper'].keys()), value='500 kcmil', description='Size:', layout=layout, style={'description_width': 'initial'})
length = widgets.FloatText(value=25, description='Length (ft):', layout=layout, style={'description_width': 'initial'})
clearing_time = widgets.FloatText(value=0.2, description='Clearing Time (s):', layout=layout, style={'description_width': 'initial'})
distance = widgets.FloatText(value=457.2, description='Work Distance (mm):', layout=layout, style={'description_width': 'initial'})
enclosure = widgets.Dropdown(options=[('Enclosed', 1.5), ('Open Air', 1.0)], value=1.5, description='Enclosure:', layout=layout, style={'description_width': 'initial'})
gap = widgets.FloatText(value=32, description='Electrode Gap (mm):', layout=layout, style={'description_width': 'initial'})

output = widgets.Output()

def calculate_all(config, kva, voltage, impedance, material, conductor_size, length, clearing_time, distance, enclosure, gap):
    if config == '3ph':
        fla = kva * 1000 / (np.sqrt(3) * voltage)
    else:
        fla = kva * 1000 / voltage

    if_primary = fla * 100 / impedance

    z_per_foot = impedance_data[material][conductor_size] / 1000
    z_total = 2 * z_per_foot * length
    conductor_percent_z = (z_total * fla * voltage) / 1000

    if_secondary = fla * 100 / (impedance + conductor_percent_z)

    ia = 0.85 * if_secondary
    cf = enclosure
    t = clearing_time
    x = distance
    E_simplified = 4.184 * cf * (ia**0.973) * (t**0.999) / (x**2)

    try:
        log10E_ieee = -0.792 + 0.729 * np.log10(ia) - 0.391 * np.log10(distance) + 0.121 * np.log10(gap) + 0.965 * np.log10(t)
        E_ieee = 10 ** log10E_ieee
    except:
        E_ieee = float('nan')

    return f"""### Results
**Transformer FLA:** {fla:.2f} A  
**Fault Current at Transformer Terminals:** {if_primary:.0f} A  
**Fault Current at Service Entrance:** {if_secondary:.0f} A  

### Arc Flash Incident Energy
- **Simplified IEEE 1584 (J/cm²):** {E_simplified:.2f}  
- **IEEE 1584-2018 (cal/cm²):** {E_ieee:.2f}  
"""

def update_output(change=None):
    with output:
        output.clear_output()
        result = calculate_all(
            config.value, kva.value, voltage.value, impedance.value,
            material.value, conductor_size.value, length.value,
            clearing_time.value, distance.value, enclosure.value,
            gap.value
        )
        display(Markdown(result))

for w in [config, kva, voltage, impedance, material, conductor_size, length, clearing_time, distance, enclosure, gap]:
    w.observe(update_output, names='value')

# Arrange widgets in rows if desired; here we stack them for simplicity
display(config, kva, voltage, impedance, material, conductor_size, length,
        clearing_time, distance, enclosure, gap, output)
update_output()
